In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'velox.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

import pandas as pd
from velox_app.models import Horse, Measure

In [2]:
horses = Horse.objects.all()
len(horses)

7553

In [3]:
measures = Measure.objects.all()
len(measures)

11002

In [4]:
# init
horses_dict = [h.__dict__ for h in horses]
measures_dict = [m.__dict__ for m in measures]
df_horses = pd.DataFrame(horses_dict)
df_measures = pd.DataFrame(measures_dict)
df_measures.drop(columns=['_state'], inplace=True)
df_horses.drop(columns=['_state'], inplace=True)
merged_df = df_measures.merge(df_horses, how='left', left_on='horse_id', right_on='id', suffixes=('_m', '_h'))

In [5]:
# init 2
def calculate_type(row):
    days_old = row['days_old']
    sex = row['sex']
    horse_age_type = ''
    if 365 >= days_old >= 0:
        horse_age_type = 'Weanling'
    elif 670 >= days_old >= 366:
        horse_age_type = 'Yearling'
    elif 990 >= days_old >= 671:
        horse_age_type = '2YO'
    elif days_old >= 991:
        horse_age_type = 'Older'
    
    horse_type = f'{sex} {horse_age_type}'
    row['horse_type'] = horse_type
    row['horse_age_type'] = horse_age_type
    return row
    
merged_df = merged_df.apply(calculate_type, axis=1)
merged_df['biomechanics'] = merged_df['gcs_bucket'].apply(lambda x: False if x is None else True)
merged_df['cardio'] = merged_df['cardio_video'].apply(lambda x: False if x is None else True)
merged_df['biomechanics_cluster'] = merged_df['biomechanics_cluster'].astype('Int64', errors='ignore')
merged_df['cardio_cluster'] = merged_df['cardio_cluster'].astype('Int64', errors='ignore')

In [6]:
# population
population_biomechanics = merged_df[merged_df['biomechanics'] == True].drop_duplicates(subset=['horse_id'])['horse_type'].value_counts().to_dict()
population_cardio = merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])['horse_type'].value_counts().to_dict()


In [108]:
# population per elite
merged_df[merged_df['biomechanics'] == True].drop_duplicates(subset=['horse_id'])[['horse_type', 'elite']].value_counts().to_dict()
merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_type', 'elite']].value_counts().to_dict()


{('Male Yearling', 'No'): 1475,
 ('Female Yearling', 'No'): 1096,
 ('Female Older', 'No'): 707,
 ('Female Older', 'Yes'): 520,
 ('Male Yearling', 'Yes'): 219,
 ('Male Older', 'Yes'): 173,
 ('Male 2YO', 'No'): 169,
 ('Female 2YO', 'No'): 156,
 ('Female Weanling', 'No'): 115,
 ('Male Older', 'No'): 106,
 ('Male Weanling', 'No'): 102,
 ('Female Yearling', 'Yes'): 98,
 ('Male 2YO', 'Yes'): 50,
 ('Female 2YO', 'Yes'): 29,
 ('Male Weanling', 'Yes'): 5,
 ('Male ', 'No'): 3,
 ('Female Weanling', 'Yes'): 1}

In [119]:
# Cardio tab
merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_age_type', 'cardio_video_score', 'elite']].sort_values(by=['horse_age_type']).value_counts()

merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_age_type', 'cardio_cluster', 'elite']].sort_values(by=['horse_age_type']).value_counts()
merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_age_type', 'type']].sort_values(by=['horse_age_type']).value_counts()


horse_age_type  type         
Yearling        Flat             2882
Older           Flat             1492
2YO             Flat              398
Weanling        Flat              223
Older           National Hunt      14
2YO             National Hunt       6
Yearling        National Hunt       6
                Flat                3
dtype: int64

In [147]:
merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_type', 'elite']].value_counts().reset_index().reset_index().rename(columns={0: 'counts'}).columns


Index(['index', 'horse_type', 'elite', 'counts'], dtype='object')

In [195]:
dfx = merged_df[['horse_type', 'elite']].value_counts().reset_index()
dfx2 = pd.DataFrame(dfx)
dfx2.to_dict(orient='records')

[{'horse_type': 'Male Yearling', 'elite': 'No', 0: 3274},
 {'horse_type': 'Female Yearling', 'elite': 'No', 0: 2592},
 {'horse_type': 'Female Older', 'elite': 'No', 0: 1280},
 {'horse_type': 'Female Older', 'elite': 'Yes', 0: 935},
 {'horse_type': 'Male Yearling', 'elite': 'Yes', 0: 473},
 {'horse_type': 'Male Older', 'elite': 'Yes', 0: 392},
 {'horse_type': 'Male 2YO', 'elite': 'No', 0: 386},
 {'horse_type': 'Female 2YO', 'elite': 'No', 0: 351},
 {'horse_type': 'Female Weanling', 'elite': 'No', 0: 251},
 {'horse_type': 'Female Yearling', 'elite': 'Yes', 0: 215},
 {'horse_type': 'Male Weanling', 'elite': 'No', 0: 213},
 {'horse_type': 'Male Older', 'elite': 'No', 0: 165},
 {'horse_type': 'Male 2YO', 'elite': 'Yes', 0: 104},
 {'horse_type': 'Female 2YO', 'elite': 'Yes', 0: 60},
 {'horse_type': 'Male Weanling', 'elite': 'Yes', 0: 10},
 {'horse_type': 'Male ', 'elite': 'No', 0: 4},
 {'horse_type': 'Female Weanling', 'elite': 'Yes', 0: 1}]

In [200]:
counts_s = merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_type']].value_counts()
df = pd.DataFrame(counts_s).reset_index().rename(columns={0: 'counts'})
df.to_dict(orient='records')

[{'horse_type': 'Male Yearling', 'counts': 1694},
 {'horse_type': 'Female Older', 'counts': 1227},
 {'horse_type': 'Female Yearling', 'counts': 1194},
 {'horse_type': 'Male Older', 'counts': 279},
 {'horse_type': 'Male 2YO', 'counts': 219},
 {'horse_type': 'Female 2YO', 'counts': 185},
 {'horse_type': 'Female Weanling', 'counts': 116},
 {'horse_type': 'Male Weanling', 'counts': 107},
 {'horse_type': 'Male ', 'counts': 3}]

In [204]:
merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id'])[['horse_age_type', 'cardio_video_score', 'elite']].value_counts()

horse_age_type  cardio_video_score
Yearling        D                     2453
Older           D                      545
                A+                     470
2YO             D                      277
Weanling        D                      195
Yearling        C                      155
Older           C                      152
                A                      151
                B                      107
Yearling        B                       82
                A+                      81
Older           B+                      72
Yearling        B+                      38
                A                       36
2YO             A+                      34
                C                       31
                A                       22
                B                       21
Weanling        C                       19
2YO             B+                      16
Weanling        B                        5
                D                        3
Weanling        A+ 

In [212]:
dfx2 = merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id']).groupby('horse_age_type')['cardio_video_score'].value_counts()
dfx = pd.DataFrame(dfx2)

In [222]:
dfx.rename(columns={'cardio_video_score': 'counts'}).reset_index().to_dict(orient='records')

[{'horse_age_type': '', 'cardio_video_score': 'D', 'counts': 3},
 {'horse_age_type': '2YO', 'cardio_video_score': 'D', 'counts': 277},
 {'horse_age_type': '2YO', 'cardio_video_score': 'A+', 'counts': 34},
 {'horse_age_type': '2YO', 'cardio_video_score': 'C', 'counts': 31},
 {'horse_age_type': '2YO', 'cardio_video_score': 'A', 'counts': 22},
 {'horse_age_type': '2YO', 'cardio_video_score': 'B', 'counts': 21},
 {'horse_age_type': '2YO', 'cardio_video_score': 'B+', 'counts': 16},
 {'horse_age_type': 'Older', 'cardio_video_score': 'D', 'counts': 545},
 {'horse_age_type': 'Older', 'cardio_video_score': 'A+', 'counts': 470},
 {'horse_age_type': 'Older', 'cardio_video_score': 'C', 'counts': 152},
 {'horse_age_type': 'Older', 'cardio_video_score': 'A', 'counts': 151},
 {'horse_age_type': 'Older', 'cardio_video_score': 'B', 'counts': 107},
 {'horse_age_type': 'Older', 'cardio_video_score': 'B+', 'counts': 72},
 {'horse_age_type': 'Weanling', 'cardio_video_score': 'D', 'counts': 195},
 {'horse_a

In [232]:
dfx2 = merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id']).groupby(['cardio_cluster', 'elite'])['cardio_video_score' ].value_counts()
dfx = pd.DataFrame(dfx2)
df_out = dfx.rename(columns={'cardio_video_score': 'counts',}).reset_index().rename(columns={'cardio_video_score': 'score', 'cardio_cluster': 'cluster'})
df_out['cluster'] = df_out['cluster'].astype(int)
df_out.to_dict(orient='records')

[{'cluster': 1, 'elite': 'No', 'score': 'D', 'counts': 313},
 {'cluster': 1, 'elite': 'No', 'score': 'C', 'counts': 42},
 {'cluster': 1, 'elite': 'No', 'score': 'B', 'counts': 27},
 {'cluster': 1, 'elite': 'No', 'score': 'A', 'counts': 22},
 {'cluster': 1, 'elite': 'No', 'score': 'A+', 'counts': 14},
 {'cluster': 1, 'elite': 'No', 'score': 'B+', 'counts': 9},
 {'cluster': 1, 'elite': 'Yes', 'score': 'A+', 'counts': 115},
 {'cluster': 1, 'elite': 'Yes', 'score': 'A', 'counts': 33},
 {'cluster': 1, 'elite': 'Yes', 'score': 'D', 'counts': 19},
 {'cluster': 1, 'elite': 'Yes', 'score': 'B', 'counts': 18},
 {'cluster': 1, 'elite': 'Yes', 'score': 'B+', 'counts': 14},
 {'cluster': 1, 'elite': 'Yes', 'score': 'C', 'counts': 13},
 {'cluster': 2, 'elite': 'No', 'score': 'D', 'counts': 872},
 {'cluster': 2, 'elite': 'No', 'score': 'C', 'counts': 7},
 {'cluster': 2, 'elite': 'No', 'score': 'A', 'counts': 2},
 {'cluster': 2, 'elite': 'No', 'score': 'B+', 'counts': 2},
 {'cluster': 2, 'elite': 'No',

In [233]:
dfx2 = merged_df[merged_df['cardio'] == True].drop_duplicates(subset=['horse_id']).groupby(['type', 'elite'])['cardio_video_score' ].value_counts()
dfx = pd.DataFrame(dfx2)
df_out = dfx.rename(columns={'cardio_video_score': 'counts',}).reset_index().rename(columns={'cardio_video_score': 'score'})
df_out.to_dict(orient='records')

[{'type': 'Flat', 'elite': 'No', 'score': 'D', 'counts': 3118},
 {'type': 'Flat', 'elite': 'No', 'score': 'C', 'counts': 260},
 {'type': 'Flat', 'elite': 'No', 'score': 'A+', 'counts': 164},
 {'type': 'Flat', 'elite': 'No', 'score': 'B', 'counts': 132},
 {'type': 'Flat', 'elite': 'No', 'score': 'A', 'counts': 111},
 {'type': 'Flat', 'elite': 'No', 'score': 'B+', 'counts': 76},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'A+', 'counts': 416},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'D', 'counts': 344},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'A', 'counts': 100},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'C', 'counts': 95},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'B', 'counts': 79},
 {'type': 'Flat', 'elite': 'Yes', 'score': 'B+', 'counts': 48},
 {'type': 'National Hunt', 'elite': 'No', 'score': 'D', 'counts': 11},
 {'type': 'National Hunt', 'elite': 'No', 'score': 'A+', 'counts': 5},
 {'type': 'National Hunt', 'elite': 'No', 'score': 'B', 'counts': 4},
 {'type': 'National Hunt'

In [235]:
counts_s = merged_df[merged_df['biomechanics'] == True].drop_duplicates(subset=['horse_id'])[
                ['horse_type', 'elite']].value_counts()
stats_df = pd.DataFrame(counts_s).reset_index().rename(columns={0: 'counts'})
stats_df.to_dict(orient='records')

[{'horse_type': 'Male Yearling', 'elite': 'No', 'counts': 1669},
 {'horse_type': 'Female Yearling', 'elite': 'No', 'counts': 1392},
 {'horse_type': 'Female Older', 'elite': 'No', 'counts': 810},
 {'horse_type': 'Female Older', 'elite': 'Yes', 'counts': 571},
 {'horse_type': 'Male Older', 'elite': 'Yes', 'counts': 275},
 {'horse_type': 'Male Yearling', 'elite': 'Yes', 'counts': 233},
 {'horse_type': 'Male 2YO', 'elite': 'No', 'counts': 190},
 {'horse_type': 'Female 2YO', 'elite': 'No', 'counts': 152},
 {'horse_type': 'Female Weanling', 'elite': 'No', 'counts': 131},
 {'horse_type': 'Female Yearling', 'elite': 'Yes', 'counts': 111},
 {'horse_type': 'Male Weanling', 'elite': 'No', 'counts': 101},
 {'horse_type': 'Male Older', 'elite': 'No', 'counts': 93},
 {'horse_type': 'Male 2YO', 'elite': 'Yes', 'counts': 43},
 {'horse_type': 'Female 2YO', 'elite': 'Yes', 'counts': 28},
 {'horse_type': 'Male Weanling', 'elite': 'Yes', 'counts': 5}]

In [11]:
cardio_bio_df = merged_df[['horse_type', 'elite', 'date_of_measure', 'horse_id', 'prob_cardio_bio_model_score']]

In [14]:
cardio_bio_df_clean = cardio_bio_df[~cardio_bio_df['prob_cardio_bio_model_score'].isnull()].drop_duplicates(subset=['horse_id', 'date_of_measure']).drop(columns=['horse_id', 'date_of_measure'])

In [17]:
cardio_bio_df_clean.groupby(['horse_type', 'elite', 'prob_cardio_bio_model_score'], as_index=False).value_counts().to_dict(orient='records')

[{'horse_type': 'Female 2YO',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'C',
  'count': 60},
 {'horse_type': 'Female 2YO',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'D',
  'count': 99},
 {'horse_type': 'Female 2YO',
  'elite': 'Yes',
  'prob_cardio_bio_model_score': 'B',
  'count': 2},
 {'horse_type': 'Female 2YO',
  'elite': 'Yes',
  'prob_cardio_bio_model_score': 'C',
  'count': 6},
 {'horse_type': 'Female 2YO',
  'elite': 'Yes',
  'prob_cardio_bio_model_score': 'D',
  'count': 7},
 {'horse_type': 'Female Older',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'B',
  'count': 77},
 {'horse_type': 'Female Older',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'B+',
  'count': 19},
 {'horse_type': 'Female Older',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'C',
  'count': 216},
 {'horse_type': 'Female Older',
  'elite': 'No',
  'prob_cardio_bio_model_score': 'D',
  'count': 370},
 {'horse_type': 'Female Older',
  'elite': 'Yes',
  'prob_cardio_bio_model_score

In [28]:
def get_tabular_age_measures_stats(df: pd.DataFrame, prob_field_name, field_type) -> list[dict]:
    dfx = df[[field_type, 'elite', 'date_of_measure', 'horse_id', prob_field_name]]
    dfx = dfx[~dfx[prob_field_name].isnull()].drop_duplicates(subset=['horse_id', 'date_of_measure']).drop(columns=['horse_id', 'date_of_measure'])
    data = dfx.groupby([field_type, 'elite', prob_field_name], as_index=False).value_counts().to_dict(orient='records')
    return data

cardio_biomechanics_tabular_data = get_tabular_age_measures_stats(merged_df, 'prob_cardio_bio_model_score', 'horse_type')
biomechanics_tabular_data = get_tabular_age_measures_stats(merged_df, 'prob_bio_model_score', 'type')

{'horse_type': 'Female 2YO', 'elite': 'No', 'score': 'C', 'counts': 60}


In [25]:
# dfxx = merged_df[['biomechanics_cluster', 'elite', 'prob_bio_model_score', 'horse_id', 'date_of_measure']]
get_tabular_age_measures_stats(merged_df, 'prob_bio_model_score', 'biomechanics_cluster')

[{'biomechanics_cluster': 0,
  'elite': 'No',
  'prob_bio_model_score': 'B',
  'count': 16},
 {'biomechanics_cluster': 0,
  'elite': 'No',
  'prob_bio_model_score': 'B+',
  'count': 1},
 {'biomechanics_cluster': 0,
  'elite': 'No',
  'prob_bio_model_score': 'C',
  'count': 14},
 {'biomechanics_cluster': 0,
  'elite': 'No',
  'prob_bio_model_score': 'D',
  'count': 11},
 {'biomechanics_cluster': 0,
  'elite': 'Yes',
  'prob_bio_model_score': 'B',
  'count': 24},
 {'biomechanics_cluster': 0,
  'elite': 'Yes',
  'prob_bio_model_score': 'B+',
  'count': 35},
 {'biomechanics_cluster': 0,
  'elite': 'Yes',
  'prob_bio_model_score': 'C',
  'count': 5},
 {'biomechanics_cluster': 0,
  'elite': 'Yes',
  'prob_bio_model_score': 'D',
  'count': 1},
 {'biomechanics_cluster': 1,
  'elite': 'No',
  'prob_bio_model_score': 'B',
  'count': 19},
 {'biomechanics_cluster': 1,
  'elite': 'No',
  'prob_bio_model_score': 'C',
  'count': 91},
 {'biomechanics_cluster': 1,
  'elite': 'No',
  'prob_bio_model_sco

In [6]:
biomechanics_df = merged_df[merged_df['biomechanics'] == True]

In [9]:
def get_scores_by_age(df: pd.DataFrame, score_field: str):
    dfx = df.drop_duplicates(subset=['horse_id'])[
        ['horse_age_type', score_field, 'elite']]
    df_out = dfx.reset_index().rename(columns={score_field: 'counts'})
    return df_out.to_dict(orient='records')

biomechanics_score_age = get_scores_by_age(biomechanics_df, 'biomechanics_video_score')
print(biomechanics_score_age)

[{'horse_age_type': 'Yearling', 'counts': 'D', 'elite': 'No', 0: 3111}, {'horse_age_type': 'Older', 'counts': 'D', 'elite': 'No', 0: 917}, {'horse_age_type': '2YO', 'counts': 'D', 'elite': 'No', 0: 379}, {'horse_age_type': 'Weanling', 'counts': 'D', 'elite': 'No', 0: 375}, {'horse_age_type': 'Older', 'counts': 'D', 'elite': 'Yes', 0: 295}, {'horse_age_type': 'Older', 'counts': 'A+', 'elite': 'Yes', 0: 166}, {'horse_age_type': 'Yearling', 'counts': 'D', 'elite': 'Yes', 0: 114}, {'horse_age_type': 'Older', 'counts': 'A+', 'elite': 'No', 0: 79}, {'horse_age_type': 'Yearling', 'counts': 'C', 'elite': 'No', 0: 69}, {'horse_age_type': 'Older', 'counts': 'C', 'elite': 'Yes', 0: 63}, {'horse_age_type': 'Older', 'counts': 'B', 'elite': 'Yes', 0: 61}, {'horse_age_type': 'Older', 'counts': 'C', 'elite': 'No', 0: 53}, {'horse_age_type': 'Older', 'counts': 'A', 'elite': 'Yes', 0: 45}, {'horse_age_type': 'Yearling', 'counts': 'A+', 'elite': 'No', 0: 42}, {'horse_age_type': 'Older', 'counts': 'B+', '

In [68]:
df = biomechanics_df
score_field = 'biomechanics_video_score'
dfx = df.drop_duplicates(subset=['horse_id'])[
        ['horse_age_type', score_field, 'elite']]
# dfx.pivot(values='elite', columns=[ 'horse_age_type', score_field]).to_dict(orient='records')
df_final = dfx.set_index(['horse_age_type', score_field]).groupby(level=[0, 1]).value_counts()

In [67]:
# df_final.unstack()
# {level: df.xs(level) for level in df.index.levels[0]}
# [df.xs(level) for level in df.index.levels[0]]
# [for df.xs(df.index.levels[0])
df.xs(df.index.levels[0][0])

biomechanics_video_score  elite
D                         No       1
dtype: int64

In [ ]:
def get_stats(df: pd.DataFrame, grouped_field, score_field,):
    """
    :param df:
    :param type_field: horse_age_type for example
    :param score_field: biomechanics_video_score for example
    :return:
    """
    dfx = df.drop_duplicates(subset=['horse_id'])[[grouped_field, score_field, 'elite']]
    dfx = dfx.set_index([grouped_field, score_field]).groupby(level=[0, 1]).value_counts()
    data = {}
    for l1 in dfx.index.levels[0]:
        if not l1:
            continue
        l2_data = {}
        for l2 in dfx.index.levels[1]:
            try:
                d = dfx.xs(l1).xs(l2).to_dict()
                l2_data[l2]=d
            except KeyError as e:
                pass
        data[l1] = l2_data
    return data
from pprint import pprint
grouped_field = 'horse_age_type'
# pprint(get_stats(biomechanics_df, 'horse_age_type', 'biomechanics_video_score'))
cardio_df = merged_df[merged_df['cardio'] == True]
# grouped_field = 'cardio_cluster'
# score_field = 'cardio_video_score'
# dfx = cardio_df
# dfx = df.drop_duplicates(subset=['horse_id'])[[grouped_field, score_field, 'elite']]
# dfx = dfx.set_index([grouped_field, score_field]).groupby(level=[0, 1]).value_counts()
# dfx.index.levels
cardio_score_cluster = get_stats(cardio_df, 'cardio_cluster', 'cardio_video_score')
pprint(cardio_score_cluster)

In [119]:
def get_scores_by_type(df: pd.DataFrame, score_column) -> list[dict]:
    dfx2 = df.drop_duplicates(subset=['horse_id']).groupby(['type', 'elite'])[
        score_column].value_counts()
    dfx = pd.DataFrame(dfx2)
    df_out = dfx.rename(columns={score_column: 'counts', }).reset_index().rename(
        columns={score_column: 'score'})
    return df_out.to_dict(orient='records')

cardio_score_type = get_scores_by_type(cardio_df, 'cardio_video_score')
print(cardio_score_type)

[{'type': 'Flat', 'elite': 'No', 'score': 'D', 'counts': 3327}, {'type': 'Flat', 'elite': 'No', 'score': 'A+', 'counts': 315}, {'type': 'Flat', 'elite': 'No', 'score': 'C', 'counts': 294}, {'type': 'Flat', 'elite': 'No', 'score': 'B', 'counts': 170}, {'type': 'Flat', 'elite': 'No', 'score': 'A', 'counts': 140}, {'type': 'Flat', 'elite': 'No', 'score': 'B+', 'counts': 93}, {'type': 'Flat', 'elite': 'Yes', 'score': 'A+', 'counts': 287}, {'type': 'Flat', 'elite': 'Yes', 'score': 'D', 'counts': 237}, {'type': 'Flat', 'elite': 'Yes', 'score': 'A', 'counts': 77}, {'type': 'Flat', 'elite': 'Yes', 'score': 'C', 'counts': 69}, {'type': 'Flat', 'elite': 'Yes', 'score': 'B', 'counts': 52}, {'type': 'Flat', 'elite': 'Yes', 'score': 'B+', 'counts': 35}, {'type': 'National Hunt', 'elite': 'No', 'score': 'D', 'counts': 25}, {'type': 'National Hunt', 'elite': 'No', 'score': 'A+', 'counts': 8}, {'type': 'National Hunt', 'elite': 'No', 'score': 'B', 'counts': 5}, {'type': 'National Hunt', 'elite': 'No',